In [1]:
import pyodbc
import pandas as pd
import decimal

In [2]:
string = (r'Driver={Microsoft Access Driver (*.mdb, *.accdb)};'+
          r'DBQ=C:\Users\Edoardo\Documents\repositories\sorting-problem\DbOrdinamento.accdb')
conn = pyodbc.connect(string)
cursor = conn.cursor()
print([t[2] for t in cursor.tables()])
conn.close()

['_NS_LOCALE_STAMPA', '_NS_LOCALE_STAMPA1', 'MSysAccessStorage', 'MSysAccessXML', 'MSysACEs', 'MSysComplexColumns', 'MSysIMEXColumns', 'MSysIMEXSpecs', 'MSysNameMap', 'MSysNavPaneGroupCategories', 'MSysNavPaneGroups', 'MSysNavPaneGroupToObjects', 'MSysNavPaneObjectIDs', 'MSysObjects', 'MSysQueries', 'MSysRelationships', 'MSysResources', '_NS_LOCALE_STAMPA2', 'Copia di VariabiliMacchina', 'Errori di incollamento', 'Variabili', 'VariabiliMacchina', 'Q_ControllaOrdinato', 'Q_EstraiPerOrdinamento']


In [3]:
def read_data_from_access(path, table):
    string = (r'Driver={Microsoft Access Driver (*.mdb, *.accdb)};'+
              'DBQ=' + path)
    conn = pyodbc.connect(string)
    cursor = conn.cursor()

    query = f'select * from {table}'  # Q_EstraiPerOrdinamento, Variabili
    cursor.execute(query)

    data = cursor.fetchall()
    conn.close()

    columns = [c[0] for c in cursor.description]
    types = [c[1] for c in cursor.description]

    df = pd.DataFrame([list(row) for row in data], columns=columns)
    for i in range(len(columns)):
        if types[i] in [decimal.Decimal, int]:
            df[columns[i]] = df[columns[i]].astype(float)
    return df

In [5]:
path = r'C:\Users\Edoardo\Documents\repositories\sorting-problem\DbOrdinamento.accdb'
table = 'Q_EstraiPerOrdinamento'
df = read_data_from_access(path, table)
df.Macchina.unique()

array(['Onix108', 'OnixK0', 'Diamond'], dtype=object)

In [79]:
df.head()

,Macchina,Prog,_C_for,Var_K,NumeroCom,IDRIGA,Fascia,NrCol,QTS,S,T,DescCol01,DescCol02,DescCol03,DescCol04,DescCol05,DescCol06,DescCol07,DescCol08
0,Onix108,10.0,20.0,AB90,104967.0,1.0,0910,1,S,EC1245,EE1817,##Nero,None,None,None,None,None,None,None
1,OnixK0,10.0,180.0,KB90,104750.0,1.0,0347,8,Q,IB5256,IB6406,process black U,##Nero,process magenta U,process yellows U,412 U (Grigio),7531 U (Grigio),180 U (Bordeaux),process cyan U
2,Diamond,11.0,0.0,DXD5,104885.0,2.0,1000,4,S,BC0089,BC0089,485 C (Rosso),348 C (Verde),cool gray 6 C (Grigio),##Nero,None,None,None,None
3,Diamond,12.0,40.0,KB75,104942.0,1.0,0330,2,S,8N036,BI2633,##Blu,##Azzurro,None,None,None,None,None,None
4,Diamond,13.0,80.0,KB75,104979.0,1.0,0327,4,S,IA1921,IA2330,@@Campione Blu,@@Campione Blu,@@Campione Azzurro,@@Campione Arancione,None,None,None,None


In [74]:
df.Macchina.unique()

array(['Diamond', 'OnixK0', 'Onix108'], dtype=object)

In [58]:
string = (r'Driver={Microsoft Access Driver (*.mdb, *.accdb)};'+
          r'DBQ=C:\Users\Edoardo\Documents\repositories\sorting-problem\DbOrdinamento.accdb')
conn = pyodbc.connect(string)
cursor = conn.cursor()
cursor.execute('Select * from Q_EstraiPerOrdinamento')

new_cols = []
for col in cursor.description:
    column_name = col[0]
    column_type = col[1]
    if column_type == str:
        column_type = 'varchar'
    elif column_type == int:
        column_type = 'integer'
    elif column_type == decimal.Decimal:
        column_type = 'double'
    new_cols.append(column_name + ' ' + column_type)


In [61]:
query = f"CREATE TABLE Test_table({', '.join(new_cols)})"
cursor.execute(query)
conn.commit()
conn.close()

In [77]:
string = (r'Driver={Microsoft Access Driver (*.mdb, *.accdb)};'+
          r'DBQ=C:\Users\Edoardo\Documents\repositories\sorting-problem\DbOrdinamento.accdb')
conn = pyodbc.connect(string)
cursor = conn.cursor() 
query1 = "update Q_EstraiPerOrdinamento set prog=10 where macchina='Diamond' and prog=10.0"
cursor.execute(query1)
conn.commit()
conn.close()